## Предсказание законтрактованности путём агрегации данных о предыдущих поставках

Мы проведём сравнение двух методов:
1) Усреднение параметров поставок (таких как процент опоздания по поставкам и изменение качества)
2) Усреднение эмбеддингов предыдущих поставок, полученных через LSTM автоэнкодер

In [11]:
import numpy as np
import pandas as pd

from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

import sys
import os
sys.path.append(os.path.abspath('..'))
from data.feature_generation import pipeline
from figures.plots import show_embeddings
from src.metrics import mapk

import warnings
warnings.filterwarnings('ignore')

In [14]:
data = pipeline()
data.head(2)

,supplier,id,spec_date,delivery_period_end,bids_contracted,is_late,is_underweight,is_poorquality,supplier_status,option,...,6,7,8,9,10,11,12,13,14,15
0,50 ЛЕТ ОКТЯБРЯ ООО,"50 ЛЕТ ОКТЯБРЯ ООО_Юг Руси, АО_2022-12-19",2022-12-19,2022-12-25,1.0,0.0,1.0,1.0,1.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,50 ЛЕТ ОКТЯБРЯ ООО,"50 ЛЕТ ОКТЯБРЯ ООО_Юг Руси, АО_2023-01-24",2023-01-24,2023-02-12,1.0,0.0,0.0,0.0,1.0,15.0,...,-0.610757,0.14691,-0.80265,0.722523,-0.47786,-0.424774,0.521571,-0.012061,0.157792,0.696974


In [27]:
"""
Разделение набора данных
"""

split_point = np.quantile(data['spec_date'], 0.95)
train_spec = data.loc[data['spec_date'] < split_point]
test_spec = data.loc[data['spec_date'] >= split_point]

X_train = train_spec.drop(['bids_contracted', 'is_late', 'is_underweight', 'is_poorquality'], axis=1)
y_train = train_spec['bids_contracted']

X_test = test_spec.drop(['bids_contracted', 'is_late', 'is_underweight', 'is_poorquality'], axis=1)
y_test = test_spec['bids_contracted']

In [29]:
"""
Предсказание законтрактованности (Простые агрегации)
"""

def estimate(model, X_test, y_test):
    accuracy = accuracy_score(y_test, model.predict(X_test))
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    print(f'Accuracy: {accuracy}\nROC AUC: {roc_auc}')

features = [['supplier_lateness', 'supplier_underweight', 'supplier_price_change'],
            [str(i) for i in range(16)],
            ['delivery_length', 'mean_delivery_length', 'delivery_length_diff', 'mean_volume', 'volume_diff', 'conversion']]

print('Бустинг (Простые агрегации):')
model = CatBoostClassifier(verbose = 0, random_state=42).fit(X_train[features[0]], y_train)
estimate(model, X_test[features[0]], y_test)

Бустинг (Простые агрегации):
Accuracy: 0.5959302325581395
ROC AUC: 0.6812189433120374


In [30]:
"""
Предсказание законтрактованности (агрегация эмбеддингов)
"""

print('Бустинг (агрегация эмбеддингов):')
model = CatBoostClassifier(verbose = 0, random_state=42).fit(X_train[features[1]], y_train)
estimate(model, X_test[features[1]], y_test)

Бустинг (агрегация эмбеддингов):
Accuracy: 0.6627906976744186
ROC AUC: 0.7093432038670857


In [31]:
def split_by_day(X, y, features, model):
    actual = []
    predicted = []
    
    days_indexes = X.groupby('spec_date').apply(lambda x: x.index)
    for day in days_indexes:
        actual.append(list(y.loc[day].values))
        predicted.append(list(model.predict(X[features].loc[day])))
    
    return actual, predicted

In [32]:
print('Бустинг (Простые агрегации):')
model = CatBoostClassifier(verbose = 0, random_state=42).fit(X_train[features[0]], y_train)
actual, predicted = split_by_day(X_test, y_test, features[0], model)
print(f'{mapk(actual, predicted)}')

Бустинг (Простые агрегации):
0.30055467372134037


In [33]:
print('Бустинг (агрегация эмбеддингов):')
model = CatBoostClassifier(verbose = 0, random_state=42).fit(X_train[features[1]], y_train)
actual, predicted = split_by_day(X_test, y_test, features[1], model)
print(f'{mapk(actual, predicted)}')

Бустинг (агрегация эмбеддингов):
0.30356261022927694


## Комбинирование переменных

In [34]:
"""
Предсказание законтрактованности (Простые агрегации + агрегации спецификаций)
"""

def estimate(model, X_test, y_test):
    accuracy = accuracy_score(y_test, model.predict(X_test))
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    print(f'Accuracy: {accuracy}\nROC AUC: {roc_auc}')

print('Бустинг (Простые агрегации):')
model = CatBoostClassifier(verbose = 0, random_state=42).fit(X_train[features[0] + features[2]], y_train)
estimate(model, X_test[features[0] + features[2]], y_test)

Бустинг (Простые агрегации):
Accuracy: 0.6453488372093024
ROC AUC: 0.7351688469729237


In [35]:
"""
Предсказание законтрактованности (Простые агрегации + агрегации спецификаций)
"""

def estimate(model, X_test, y_test):
    accuracy = accuracy_score(y_test, model.predict(X_test))
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    print(f'Accuracy: {accuracy}\nROC AUC: {roc_auc}')

print('Бустинг (агрегация эмбеддингов):')
model = CatBoostClassifier(verbose = 0, random_state=42).fit(X_train[features[1] + features[2]], y_train)
estimate(model, X_test[features[1] + features[2]], y_test)

Бустинг (агрегация эмбеддингов):
Accuracy: 0.6773255813953488
ROC AUC: 0.7353040597640537


In [36]:
print('Бустинг (Простые агрегации):')
model = CatBoostClassifier(verbose = 0, random_state=42).fit(X_train[features[0]], y_train)
actual, predicted = split_by_day(X_test, y_test, features[0] + features[2], model)
mapk(actual, predicted)

Бустинг (Простые агрегации):


0.30055467372134037

In [37]:
print('Бустинг (агрегация эмбеддингов):')
model = CatBoostClassifier(verbose = 0, random_state=42).fit(X_train[features[1]], y_train)
actual, predicted = split_by_day(X_test, y_test, features[1] + features[2], model)
mapk(actual, predicted)

Бустинг (агрегация эмбеддингов):


0.30356261022927694